<a href="https://colab.research.google.com/github/sarabdar/keras/blob/main/Custom_Training_loop_in_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import warnings
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Input
from tensorflow.keras.callbacks import Callback
import numpy as np

# Suppress all Python warnings
warnings.filterwarnings('ignore')

# Set TensorFlow log level to suppress warnings and info messages
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# Step 1: Set Up the Environment
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [2]:
# Step 2: Define the Model

model = Sequential([
    Flatten(input_shape=(28, 28)),
    Dense(128, activation='relu'),
    Dense(10)
])

In [5]:
# Step 3: Define Loss Function and Optimizer

loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
optimizer = tf.keras.optimizers.Adam()


accuracy_metric = tf.keras.metrics.SparseCategoricalAccuracy()  # Metric to track accuracy during training


In [6]:
# Step 4: Implement the Custom Training Loop

epochs = 2
# train_dataset = train_dataset.repeat(epochs)
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train)).batch(32)
for epoch in range(epochs):
    print(f'Start of epoch {epoch + 1}')

    for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
        with tf.GradientTape() as tape:  # GradientTape is use to record every calculation perform.
            logits = model(x_batch_train, training=True)  # Forward pass
            loss_value = loss_fn(y_batch_train, logits)  # Compute loss


        # Now that we have record of calculations (model outputs and loss)
        # We can compute gradients and update weights
        grads = tape.gradient(loss_value, model.trainable_weights)
        optimizer.apply_gradients(zip(grads, model.trainable_weights))

        # Update the accuracy metric
        accuracy_metric.update_state(y_batch_train, logits)

        # Log the loss and accuracy every 200 steps
        if step % 200 == 0:
            print(f'Epoch {epoch + 1} Step {step}: Loss = {loss_value.numpy()} Accuracy = {accuracy_metric.result().numpy()}')

    # Reset the metric at the end of each epoch
    accuracy_metric.reset_state()

Start of epoch 1
Epoch 1 Step 0: Loss = 0.120081327855587 Accuracy = 0.96875
Epoch 1 Step 200: Loss = 0.1890629678964615 Accuracy = 0.9480721354484558
Epoch 1 Step 400: Loss = 0.11970341205596924 Accuracy = 0.9456047415733337
Epoch 1 Step 600: Loss = 0.07614672183990479 Accuracy = 0.9485753178596497
Epoch 1 Step 800: Loss = 0.09109409898519516 Accuracy = 0.9504915475845337
Epoch 1 Step 1000: Loss = 0.39542412757873535 Accuracy = 0.9507992267608643
Epoch 1 Step 1200: Loss = 0.1787436455488205 Accuracy = 0.951654851436615
Epoch 1 Step 1400: Loss = 0.17438732087612152 Accuracy = 0.9522216320037842
Epoch 1 Step 1600: Loss = 0.17052042484283447 Accuracy = 0.9521978497505188
Epoch 1 Step 1800: Loss = 0.16195276379585266 Accuracy = 0.9534980654716492
Start of epoch 2
Epoch 2 Step 0: Loss = 0.04762498289346695 Accuracy = 1.0
Epoch 2 Step 200: Loss = 0.07808659225702286 Accuracy = 0.9710820913314819
Epoch 2 Step 400: Loss = 0.08706273883581161 Accuracy = 0.9685162305831909
Epoch 2 Step 600: Los